In [ ]:
# HOME
home = '/Users/mahtin'
home = '/Users/martinscience'

import os
import matplotlib as mpl
import matplotlib.pyplot as plt
# use seaborn plotting defaults
import seaborn as sns; sns.set()

# get the code
import sys
sys.path.append('../scripts')
from codeCNV.plot import plot_genomic, plot_snp2, plot_snp, plot_2d, plot_3d, add_chrom_labels, set_ticks, make_color_chroms, sort_df, get_chrom_df
from script_utils import show_output

# load the config
# edit config directly in yaml file
import yaml
config_file = '../config/config_devel.yaml'
def get_config(config_file, param):
        with open(config_file) as file:
        # The FullLoader parameter handles the conversion from YAML
        # scalar values to Python the dictionary format
            config = yaml.load(file, Loader=yaml.FullLoader)['CNV'][param]
        return config
config = get_config(config_file, 'cluster')

# load the paths
wes_path = f"{home}/Dropbox/Icke/Work/somVar/AMLMono7/WESData"
plot_path = f'{wes_path}/CNVplots/cluster'

###  get the sample
+ from the sample_df### visualize the data

In [ ]:
def get_sample(sample):
    s = sample.replace("_", "")
    sample_df = pd.read_excel(os.path.join(wes_path, "cluster_samples.xlsx")).set_index('sample')
    return sample_df.loc[s, "file"]

get_sample("04A")

In [ ]:
sample = "04A"
cluster_df = pd.read_csv(get_sample(sample), sep='\t')
cluster_df

In [ ]:
log2 = dict(
        title='log2ratio',
        plot_type='scatter',   # ['line', 'scatter']
        data='log2ratio',
        plot_args=dict(
            linewidth=0.50,
            color='black',
            s=1,
            alpha=.9
        )
    )

log2mean = dict(
        title='rollinglog2ratio',
        plot_type='line',   # ['line', 'scatter']
        data='log2ratiomean',
        plot_args=dict(
            linewidth=.5,
            color='yellow',
            alpha=.7
        )
    )

vaf = dict(
        title='VAF',
        plot_type='scatter',  # ['line', 'scatter']
        data='VAF',
        plot_args=dict(
            s=.5,
            color='black',
            alpha=.7
        )
    )

vafC = dict(
        title='VAF',
        plot_type='scatter',  # ['line', 'scatter']
        data='VAF',
        plot_args=dict(
            s=1,
            c='snpCNVcore',
            cmap="seismic",
            alpha=.7
        )
    )
absvaf = dict(
        title='absVAF',
        plot_type='scatter',  # ['line', 'scatter']
        data='absVAF',
        plot_args=dict(
            s=1,
            c='dbscan',
            alpha=.7
        )
    )

fig_params = dict(
    figsize=(24,8),
    colormap='coolwarm_r',
    color_chroms=True,
    ylim=(0,1),
    cov_offset=.1,  # how much log2ratio=0 is shifted above SNP-data
    cov_height=.5,
    label_size=13
)

fig, _, _, _ = plot_snp2(cluster_df, snp_plots=[vaf], cov_plots=[log2,log2mean], blocks=["snpCNV"], chroms='all', region='', **fig_params)

### expand plot_snp functionality to showing clusters

In [ ]:
def plot_snp2(df, snp_plots=[], cov_plots=[], blocks=[], chroms='all', cov_offset=.25, cov_height=0.5, color_chroms=True, colormap='coolwarm_r', region='', label_size=12, figsize=(20, 4), ylim=(-1, 1)):

    MAXLOG2RATIO = 2.5
    #### DATA MANGELING ##########
    # get cols for rearranging
    org_cols = list(df.columns)

    # sort the df
    df = sort_df(df)
    # reduce the df to the selected chromosomes
    if region:
        chrom, start, end = extract_pos(region)
        df = df.query('Chr == @chrom and @start <= Pos <= @end')
    elif chroms != 'all':
        df = df.query('Chr in @chroms')

    # get the chrom_df for collapsing the
    chrom_df = get_chrom_df(df)
    df = df.merge(chrom_df.loc[:, 'dif'], on='Chr')
    df['PlotPos'] = df['FullExonPos'] - df['dif']
    # rearrange the df as return value
    new_cols = org_cols[:4] + ['PlotPos'] + org_cols[4:]
    df = df.loc[:, new_cols]

    #########################
    ######## PLOTTING #######
    # plot the figure
    fig, ax = plt.subplots(figsize=figsize)

    # set the x-axis limits
    _ = ax.set_xlim(0, df['PlotPos'].max())

    # PLOT COV Data
    if len(cov_plots):
        scale_factor = cov_height / (MAXLOG2RATIO + 1)
        offset = 1 + scale_factor + cov_offset

        ylim = (ylim[0], ylim[1] + cov_offset + cov_height)

        for plot in cov_plots:
            # normalize the coverage data:
            # 2.5 is the approx max log2ratio (LOH to 8N)

            df[plot['data']] = df[plot['data']] * scale_factor + offset
            
            minus, zero, one = [c * scale_factor + offset for c in [-1, 0, 1]]
            if plot['plot_type'] == 'line':
                plot = ax.plot(df['PlotPos'], df[plot['data']],
                               **plot['plot_args'])
            elif plot['plot_type'] == 'scatter':
                # highjack plot_args
                pa = plot['plot_args']
                if "c" in pa:
                    pa['c'] = df[pa['c']]
                if "s" in pa:
                    if isinstance(pa['s'], str):
                        pa['s'] = df[pa['s']] * 20 + 1
                plot = ax.scatter(df['PlotPos'], df[plot['data']], **pa)

    ######## plot the SNP graphs #######
    for plot in snp_plots:
        if plot['plot_type'] == 'line':
            plot = ax.plot(df['PlotPos'], df[plot['data']],
                           **plot['plot_args'])
        elif plot['plot_type'] == 'scatter':
            # highjack plot_args with
            pa = plot['plot_args']
            if "c" in pa:
                pa['c'] = df[pa['c']]
            if "s" in pa:
                if isinstance(pa['s'], str):
                    pa['s'] = df[pa['s']] * 20 + 1
            plot = ax.scatter(df['PlotPos'], df[plot['data']], **pa)

    _ = ax.set_ylim(ylim)
    # add the color chroms
    _ = make_color_chroms(ax, chrom_df, color_chroms,
                          ylimits=ax.get_ylim(), colormap=colormap)

    ######## LABELS ###################
    # set the axis labels
    _ = ax.set_xlabel('genomic coords', fontsize=1.25*label_size)
    # quick fix for one y-label
    _ = ax.set_ylabel(' / '.join([plot['title']
                                  for plot in snp_plots]), fontsize=1.25*label_size)
    
    ######## BLOCKS ##################
    for col in blocks:
        if col.startswith("snp"):
            df.loc[df[col] > 0, col] = 0.5
            df.loc[df[col] == 0, col] = -10
            plot = ax.scatter(df['PlotPos'], df[col], s=20, color='green')
    if col.startswith("cov"):
            df.loc[df[col] > 0, col] = offset
            df.loc[df[col] == 0, col] = -10
            plot = ax.scatter(df['PlotPos'], df[col], s=20, color='blue')

    ######## CHROM LABELS #############
    add_chrom_labels(ax, chrom_df, ax.get_ylim())

    ####### X-AXIS ####################
    # set major ticks and grid for chrom

    ax = set_ticks(ax, df, chrom_df, label_size=label_size)
        # set helper lines
    _ = ax.axhline(y=1, c='k', lw=2, ls="-")
    _ = ax.axhline(y=.5, c='k', lw=1.5, alpha=0.5, ls="--")
    
    _ = ax.axhline(y=minus, c='k', lw=1.5, alpha=0.5, ls="--")
    _ = ax.axhline(y=zero, c='k', lw=1.5, alpha=0.5, ls="-")
    _ = ax.axhline(y=one, c='k', lw=1.5, alpha=0.5, ls="--")
    # return fig and ax for further plotting and return edited dataframe
    return fig, ax, df, chrom_df

In [ ]:
fig, _, _, _ = plot_snp2(cluster_df, snp_plots=[vaf], cov_plots=[log2,log2mean], blocks=["snpCNVcore", "covCNVcore"], chroms='all', region='', **fig_params)

In [ ]:
cluster_df.query("covCNV >0")['Chr'].unique()